In [1]:
import fedci

No GPU automatically detected. Setting SETTINGS.GPU to 0, and SETTINGS.NJOBS to cpu_count.
INFO:rpy2.situation:cffi mode is CFFI_MODE.ANY
INFO:rpy2.situation:R home found: /opt/homebrew/Caskroom/miniforge/base/envs/promotion/lib/R
INFO:rpy2.situation:R library path: 
INFO:rpy2.situation:LD_LIBRARY_PATH: 
INFO:rpy2.rinterface_lib.embedded:Default options to initialize R: rpy2, --quiet, --no-save
INFO:rpy2.rinterface_lib.embedded:R is already initialized. No need to initialize.


In [2]:
import polars as pl
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

In [21]:
import numpy as np
from tqdm.notebook import tqdm

In [3]:
TOTAL_SAMPLES = 10_000

In [12]:
# DGP 2

possible_dags = [
    "pdsep_g",
    "collider",
    "fork",
    "chain4",
    "descColl",
    "2descColl",
    "iv"
]

def get_sample_data(dag_type, num_clients, num_samples, num_vars):
    with (ro.default_converter + pandas2ri.converter).context():
        ro.r['source']('./app/scripts/example_data.r')
        get_example_data_f = ro.globalenv['get_example_data']

        result = get_example_data_f(dag_type, num_clients, num_samples, num_vars)
        
    return result

# TOTAL_FEATURES cant be set for this call right now, because the true graph is fixed
data = get_sample_data(possible_dags[3], 1, TOTAL_SAMPLES, 4)

data = list(data.items())[0][1]

In [22]:
servers = {}

servers['1 client'] = fedci.Server({0: fedci.Client(data)})

clients = {i:fedci.Client(chunk) for i,chunk in enumerate(np.array_split(data, 3))}
servers['3 clients'] = fedci.Server(clients)

#clients = {i:fedci.Client(chunk) for i,chunk in enumerate(np.array_split(data, 10))}
#servers['5 clients'] = fedci.Server(clients)

'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.


In [24]:
for server in servers.values():
    server.run_tests()

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from pgmpy.estimators import CITests
from scipy import stats

In [ ]:
class EmptyLikelihoodRatioTest(fedci.LikelihoodRatioTest):
    def __init__(self, y_label, x_label, s_labels, p_val):
        self.y_label = y_label
        self.x_label = x_label
        self.s_labels = s_labels
        self.p_val = p_val

In [26]:
testing_rounds = {k:v.testing_engine.finished_rounds for k,v in servers.items()}

possible_tests = set([(t.y_label, t.x_label, tuple(sorted(t.s_labels)), t.p_val) for t in fedci.get_likelihood_tests(testing_rounds['1 client'])])
len(possible_tests)

48

In [60]:
ground_truth_tests = []

for test in possible_tests:
    if len(test[2]) > 0:
        #v0 = data[test[0]].values
        #v1 = data[test[1]].values
        #s = data[list(test[2])].values
        #p0 = test[3]
        #p1 = citest(v0, v1, s, test_args={'statistic': 'ksg_cmi', 'n_jobs': 8})
        
        _, p1 = CITests.pearsonr(test[1], test[0], list(test[2]), data, boolean=False)
    else:
        v0 = data[test[0]]
        v1 = data[test[1]]
        correlation, p1 = stats.pearsonr(v0, v1)
        
    p1 = round(p1,4)
    
    ground_truth_tests.append(EmptyLikelihoodRatioTest(test[0], test[1], list(test[2]), p1))

In [61]:
likelihood_tests = {k:fedci.get_likelihood_tests(v) for k,v in testing_rounds.items()}

In [62]:
p_value_comparison = {k:[] for k in likelihood_tests.keys()}
missing_test = {k:0 for k in likelihood_tests.keys()}

In [63]:
for test in ground_truth_tests:
    for k, likelihood_test in likelihood_tests.items():
        matching_test = [t for t in likelihood_tests[k] if t.y_label == test.y_label and t.x_label == test.x_label and sorted(t.s_labels) == sorted(test.s_labels)]
        if len(matching_test) == 0:
            print(f'No matching test in {k} for {test}')
            missing_test[k] += 1
        assert len(matching_test) == 1
        matching_test = matching_test[0]
        
        # if matching_test.p_val != test.p_val:
        #     print('-'*10)
        #     print(test)
        #     print(matching_test)
        
        p_value_comparison[k].append((matching_test.p_val, test.p_val))

----------
LikelihoodRatioTest - y: D, x: A, S: ['B'], p: 0.8468
LikelihoodRatioTest - y: D, x: A, S: ['B'], p: 0.8467
----------
LikelihoodRatioTest - y: D, x: A, S: ['B'], p: 0.8468
LikelihoodRatioTest - y: D, x: A, S: ['B'], p: 0.2318
----------
LikelihoodRatioTest - y: D, x: A, S: ['B', 'C'], p: 0.6682
LikelihoodRatioTest - y: D, x: A, S: ['B', 'C'], p: 0.3801
----------
LikelihoodRatioTest - y: C, x: A, S: ['B', 'D'], p: 0.0896
LikelihoodRatioTest - y: C, x: A, S: ['B', 'D'], p: 0.5621
----------
LikelihoodRatioTest - y: A, x: D, S: ['B'], p: 0.8468
LikelihoodRatioTest - y: A, x: D, S: ['B'], p: 0.8467
----------
LikelihoodRatioTest - y: A, x: D, S: ['B'], p: 0.8468
LikelihoodRatioTest - y: A, x: D, S: ['B'], p: 0.2318
----------
LikelihoodRatioTest - y: B, x: D, S: ['A', 'C'], p: 0.4265
LikelihoodRatioTest - y: B, x: D, S: ['A', 'C'], p: 0.4264
----------
LikelihoodRatioTest - y: B, x: D, S: ['A', 'C'], p: 0.4265
LikelihoodRatioTest - y: B, x: D, S: ['A', 'C'], p: 0.432
---------

In [56]:
sum([abs(a-b) < 0.0001 for a,b in p_value_comparison['3 clients']]) / len(p_value_comparison['3 clients'])

0.6666666666666666

In [69]:
def count_correct_alpha_thresholdings(data, alpha):
    c = sum([1 for a,b in data if (a < alpha and b < alpha) or (a > alpha and b > alpha)]) / len(data)
    return c

def count_correct_pval(data, tolerance=1e-4):
    c = sum([1 for a,b in data if abs(a-b)<tolerance]) / len(data)
    return c

In [72]:
count_correct_alpha_thresholdings(p_value_comparison['3 clients'], 0.05)

1.0

In [74]:
count_correct_pval(p_value_comparison['3 clients'], 1e-4)

0.6666666666666666

In [ ]:
# TODO: gather all stats
# TODO: write all stats to csv for evaluation later on
# TODO: make easily loopable (single func)